# Recycling Analysis — Python

Estimates gamma(t) from bulk LF data, then applies recycling correction to single-cell mTRAQ ratios.
Mirrors `R/Recycling_analysis.R`.

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.optimize import minimize_scalar, curve_fit
from joblib import Parallel, delayed
%matplotlib inline

import quantqc as qqc
from quantqc.normalize import normalize_reference_vector
from quantqc.miceotope import miceotope_protein_collapse

# Paths
path_raw  = '/Users/andrewleduc/Desktop/jmod_raw/'
data_out  = os.path.normpath(os.path.join(os.getcwd(), '../data_out_temp')) + '/'
lf_path   = '/Users/andrewleduc/Desktop/Github/Kevin_PTI/report.parquet'

---
## 1. Helper functions

In [ ]:
def tls(vect1, vect2):
    """Total Least Squares slope via SVD. Matches R TLS()."""
    vect1 = np.array(vect1, dtype=float)
    vect2 = np.array(vect2, dtype=float)
    vect1[~np.isfinite(vect1)] = np.nan
    vect2[~np.isfinite(vect2)] = np.nan

    int_x = np.nanmean(vect1)
    int_y = np.nanmean(vect2)
    v1 = vect1 - int_x
    v2 = vect2 - int_y

    mat = np.column_stack([v1, v2])
    mat = mat[~np.isnan(mat).any(axis=1)]

    _, _, Vt = np.linalg.svd(mat)
    V = Vt.T
    slope = V[0, 0] / V[1, 0]
    return slope, (int_x, int_y)


def L_function(alpha, L0, a, b, t):
    """Analytic solution for labelled fraction after recycling correction. Matches R L_function()."""
    c = 0.5
    beta = L0 * alpha
    num = (
        a * beta * c
        - 2 * alpha * beta * c
        + b * beta * c
        + alpha * beta * c * np.exp((-a + alpha) * t)
        - b * beta * c * np.exp((-a + alpha) * t)
        - a * beta * c * np.exp((alpha - b) * t)
        + alpha * beta * c * np.exp((alpha - b) * t)
        - a * alpha * L0
        + alpha ** 2 * L0
        + a * b * L0
        - alpha * b * L0
    )
    denom = (-a + alpha) * (alpha - b) * np.exp(alpha * t)
    return num / denom


def objective_function(alpha, L_measured, L0, a, b, t):
    """RSS between predicted and measured L for a single peptide × cell."""
    L_predicted = L_function(alpha, L0, a, b, t)
    return (L_predicted - L_measured) ** 2


def _optimise_column(j, L_mat, L0_mat, a, b, t):
    """Optimise alpha for all peptides in column j (one cell). Used by joblib."""
    n_rows = L_mat.shape[0]
    col_res = np.full(n_rows, np.nan)
    for i in range(n_rows):
        L_meas = L_mat[i, j]
        L0_val = L0_mat[i, j]
        if np.isfinite(L_meas) and np.isfinite(L0_val):
            try:
                res = minimize_scalar(
                    objective_function,
                    bounds=(0, 10),
                    method='bounded',
                    args=(L_meas, L0_val, a, b, t),
                )
                col_res[i] = res.x
            except Exception:
                pass
    return col_res


def recycle_adjust_par(L_mat, L0_mat, t, a, b, ncores=4):
    """Parallel recycling correction. Returns alpha matrix same shape as L_mat.

    L_mat  : (n_peptides x n_cells) measured L / (L+H) ratio
    L0_mat : (n_peptides x n_cells) total intensity L+H
    t      : labeling time (days)
    a, b   : gamma(t) parameters from bulk LF data
    """
    n_cols = L_mat.shape[1]
    results = Parallel(n_jobs=ncores, verbose=1)(
        delayed(_optimise_column)(j, L_mat, L0_mat, a, b, t)
        for j in range(n_cols)
    )
    out = np.column_stack(results)
    return out

---
## 2. Estimate gamma(t) parameters from bulk LF data

In [ ]:
LF_data = pl.read_parquet(lf_path).to_pandas()

# Rename runs to short labels
LF_data.loc[LF_data['Run'] == '2026-01-14_2ngL2_20Th-10ms_28min_nce30', 'Run'] = 'L2'
LF_data.loc[LF_data['Run'] == '2026-01-14_2ngL1_20Th-10ms_28min_nce30', 'Run'] = 'L1'

LF_data = LF_data[LF_data['Ms1.Area'] != 0].copy()
LF_data['seqcharge'] = LF_data['Stripped.Sequence'].astype(str) + LF_data['Precursor.Charge'].astype(str)

# Keep only peptides with exactly 2 lysines
LF_data['kcount'] = LF_data['Stripped.Sequence'].str.count('K')
LF_data = LF_data[LF_data['kcount'] == 2]

# Count heavy-labeled K residues
LF_data['hcount'] = LF_data['Precursor.Id'].str.count('K_6C13-6')

HL_dat = LF_data[LF_data['hcount'] == 1].copy()
HH_dat = LF_data[LF_data['hcount'] == 2].copy()

sect = set(HL_dat['seqcharge']) & set(HH_dat['seqcharge'])
HL_dat = HL_dat[HL_dat['seqcharge'].isin(sect)]
HH_dat = HH_dat[HH_dat['seqcharge'].isin(sect)]

HL_piv = HL_dat.groupby(['seqcharge', 'Run'])['Precursor.Quantity'].median().reset_index()
HL_piv = HL_piv.pivot(index='seqcharge', columns='Run', values='Precursor.Quantity')
HH_piv = HH_dat.groupby(['seqcharge', 'Run'])['Precursor.Quantity'].median().reset_index()
HH_piv = HH_piv.pivot(index='seqcharge', columns='Run', values='Precursor.Quantity')

common_runs = HL_piv.columns.intersection(HH_piv.columns)
HL_mat = HL_piv[common_runs].values
HH_mat = HH_piv[common_runs].values

pct_Heavy = 2 / (HL_mat / HH_mat + 2)

# Complete cases only
ok = ~np.isnan(pct_Heavy).any(axis=1)
pct_Heavy_cc = pct_Heavy[ok]

print(f'Complete-case peptides: {ok.sum()}')
print(f'Runs: {list(common_runs)}')
print(f'Median pct_Heavy by run:')
for i, r in enumerate(common_runs):
    print(f'  {r}: {np.nanmedian(pct_Heavy_cc[:, i]):.4f}')

fig, axes = plt.subplots(1, len(common_runs), figsize=(5 * len(common_runs), 4))
if len(common_runs) == 1:
    axes = [axes]
for i, r in enumerate(common_runs):
    axes[i].boxplot(pct_Heavy_cc[:, i])
    axes[i].set_title(f'% Heavy — {r}')
    axes[i].set_ylabel('pct_Heavy')
plt.tight_layout()
plt.show()

In [ ]:
# Fit gamma(t) = 1 - 0.5*exp(-a*t) - 0.5*exp(-b*t)
# Observed: t=0 -> 0, t=3 -> HH_fract_3day, t=5 -> HH_fract_5day
# Update these from the medians above
HH_fract_3day = 0.5    # update if run L1 = 3 day
HH_fract_5day = 0.537  # update if run L2 = 5 day

time_obs = np.array([0.0, 3.0, 5.0])
recycle_obs = np.array([0.0, HH_fract_3day, HH_fract_5day])

def gamma_model(t, a, b):
    return 1 - 0.5 * np.exp(-a * t) - 0.5 * np.exp(-b * t)

a_init = np.log(2) / 0.7
b_init = np.log(2) / 20

popt, _ = curve_fit(gamma_model, time_obs, recycle_obs, p0=[a_init, b_init],
                    bounds=([0, 0], [np.inf, np.inf]), maxfev=10000)
a_fitted, b_fitted = popt
print(f'a_fitted = {a_fitted:.4f}, b_fitted = {b_fitted:.4f}')

t_plot = np.linspace(0, 20, 200)
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(t_plot, gamma_model(t_plot, a_fitted, b_fitted), 'b-', label='Fitted gamma(t)')
ax.scatter(time_obs, recycle_obs, color='red', zorder=5, label='Observed')
ax.set_xlabel('Time (days)')
ax.set_ylabel('Fraction heavy AA')
ax.set_title('Fitted gamma(t): free AA labeling kinetics')
ax.legend()
plt.tight_layout()
plt.show()

---
## 3. Load preprocessed QQC object

In [ ]:
with open(os.path.join(data_out, 'r1_5day_male.pkl'), 'rb') as f:
    r1_5day_male = pickle.load(f)

meta = r1_5day_male.meta_data
Raw_L = r1_5day_male.miceotopes.Raw_L
Raw_H = r1_5day_male.miceotopes.Raw_H
miceotope_cols = r1_5day_male.miceotopes.cols

print(f'Raw_L shape: {Raw_L.shape}')
print(f'Meta shape:  {meta.shape}')
print(f'Samples:     {meta["sample"].value_counts().to_dict()}')

In [ ]:
# Split cells by mouse (3-day vs 5-day labeling)
days_3_ids = meta.loc[meta['sample'] == 'mouse1', 'ID'].values
days_5_ids = meta.loc[meta['sample'] == 'mouse2', 'ID'].values

# Keep only IDs present in the miceotope matrix
days_3_ids = np.array([x for x in days_3_ids if x in miceotope_cols])
days_5_ids = np.array([x for x in days_5_ids if x in miceotope_cols])

col_idx  = {c: i for i, c in enumerate(miceotope_cols)}
idx3 = np.array([col_idx[x] for x in days_3_ids])
idx5 = np.array([col_idx[x] for x in days_5_ids])

print(f'3-day cells: {len(idx3)}, 5-day cells: {len(idx5)}')

---
## 4. Compute raw L/(L+H) ratios

In [ ]:
L3 = Raw_L[:, idx3].astype(float)
H3 = Raw_H[:, idx3].astype(float)
L5 = Raw_L[:, idx5].astype(float)
H5 = Raw_H[:, idx5].astype(float)

r1 = L3 / (L3 + H3)   # L/(L+H) for 3-day mouse
r2 = L5 / (L5 + H5)   # L/(L+H) for 5-day mouse

min_ratio_3 = 1 - HH_fract_3day
min_ratio_5 = 1 - HH_fract_5day

pct_below_3 = np.sum(r1 < min_ratio_3, where=np.isfinite(r1), initial=0) / np.sum(np.isfinite(r1))
pct_below_5 = np.sum(r2 < min_ratio_5, where=np.isfinite(r2), initial=0) / np.sum(np.isfinite(r2))
print(f'Fraction below min ratio (3-day): {pct_below_3:.3f} (~10% expected)')
print(f'Fraction below min ratio (5-day): {pct_below_5:.3f} (~20% expected)')

# NA out physically impossible ratios
r1[r1 < min_ratio_3] = np.nan
r2[r2 < min_ratio_5] = np.nan

---
## 5. Apply recycling correction (parallel)

In [ ]:
# L0_mat = total intensity (L + H)
tot3 = L3 + H3
tot5 = L5 + H5

print('Running recycling correction for 3-day mouse...')
adj_r1 = recycle_adjust_par(r1, tot3, t=3, a=a_fitted, b=b_fitted, ncores=4)

print('Running recycling correction for 5-day mouse...')
adj_r2 = recycle_adjust_par(r2, tot5, t=5, a=a_fitted, b=b_fitted, ncores=4)

# NA out boundary values (optimizer hit bounds → unreliable)
adj_r1[adj_r1 >= 10] = np.nan
adj_r1[adj_r1 <= 0]  = np.nan
adj_r2[adj_r2 >= 10] = np.nan
adj_r2[adj_r2 <= 0]  = np.nan

print(f'adj_r1 finite: {np.sum(np.isfinite(adj_r1))}')
print(f'adj_r2 finite: {np.sum(np.isfinite(adj_r2))}')

In [ ]:
# Uncorrected degradation rates: -log(L/(L+H)) / t
r1_rate = -np.log(r1) / 3
r2_rate = -np.log(r2) / 5

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(r1_rate.flatten(), adj_r1.flatten(), s=1, alpha=0.1, rasterized=True)
ax.axline((0, 0), slope=1, color='red', linestyle='--')
ax.set_xlim(0, 2.5)
ax.set_ylim(0, 2.5)
ax.set_xlabel('Uncorrected rate (day$^{-1}$)')
ax.set_ylabel('Recycling-corrected rate (day$^{-1}$)')
ax.set_title('Effect of recycling correction (3-day)')
plt.tight_layout()
plt.show()

---
## 6. Compare 3-day vs 5-day corrected rates

In [ ]:
# Filter to peptides with >= 20 observations in both groups
n_obs_r1 = np.sum(np.isfinite(adj_r1), axis=1)
n_obs_r2 = np.sum(np.isfinite(adj_r2), axis=1)
keep = (n_obs_r1 >= 20) & (n_obs_r2 >= 20)
print(f'Peptides with >=20 obs in both groups: {keep.sum()}')

adj_r1_ = adj_r1[keep]
adj_r2_ = adj_r2[keep]

mean_r1 = np.nanmean(np.log2(adj_r1_), axis=1)
mean_r2 = np.nanmean(np.log2(adj_r2_), axis=1)

ok = np.isfinite(mean_r1) & np.isfinite(mean_r2)
cor_corrected = np.corrcoef(mean_r1[ok], mean_r2[ok])[0, 1]
slope, _ = tls(mean_r1[ok], mean_r2[ok])
print(f'Corrected correlation: {cor_corrected:.4f}')
print(f'TLS slope (log2 rate 3d vs 5d): {slope:.4f}  (1/slope = {1/slope:.4f})')

X = 2 ** mean_r1[ok]
Y = 2 ** mean_r2[ok]

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, s=8, alpha=0.5)
ax.axline((0, 0), slope=1, linestyle='--', color='gray')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('3-day degradation rate (day$^{-1}$)')
ax.set_ylabel('5-day degradation rate (day$^{-1}$)')
ax.set_title(f'Recycling-corrected rates\nr = {cor_corrected:.3f}')
plt.tight_layout()
plt.show()

---
## 7. Manual slope correction

In [ ]:
adj_r1_log_tune = np.log2(adj_r1)
adj_r2_log_tune = np.log2(adj_r2)

# Compute offset: mean difference between 3-day and 5-day (in log2 space)
mean_r1_all = np.nanmean(adj_r1_log_tune[keep], axis=1)
mean_r2_all = np.nanmean(adj_r2_log_tune[keep], axis=1)
ok_all = np.isfinite(mean_r1_all) & np.isfinite(mean_r2_all)

tls_slope, _ = tls(mean_r1_all[ok_all], mean_r2_all[ok_all])
offset = np.nanmean(mean_r1_all[ok_all] - mean_r2_all[ok_all])
print(f'TLS slope: {tls_slope:.4f}, offset: {offset:.4f}')

# Apply correction to 3-day (as in R)
adj_r1_log_tune = adj_r1_log_tune * (1 / tls_slope) - offset

# Re-check
mean_r1_t = np.nanmean(adj_r1_log_tune[keep], axis=1)
mean_r2_t = np.nanmean(adj_r2_log_tune[keep], axis=1)
ok_t = np.isfinite(mean_r1_t) & np.isfinite(mean_r2_t)
cor_tuned = np.corrcoef(mean_r1_t[ok_t], mean_r2_t[ok_t])[0, 1]
print(f'Tuned correlation: {cor_tuned:.4f}')

X_t = 2 ** mean_r1_t[ok_t]
Y_t = 2 ** mean_r2_t[ok_t]
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X_t, Y_t, s=8, alpha=0.5)
ax.axline((0, 0), slope=1, linestyle='--', color='gray')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0.01, 1)
ax.set_ylim(0.01, 1)
ax.set_xlabel('3-day rate (day$^{-1}$)')
ax.set_ylabel('5-day rate (day$^{-1}$)')
ax.set_title('After manual slope correction')
plt.tight_layout()
plt.show()

---
## 8. Compare corrected vs uncorrected (reference)

In [ ]:
# Uncorrected rates for the same peptide set
r1_rate_f = r1_rate[keep]
r2_rate_f = r2_rate[keep]
mean_unc_r1 = np.nanmean(np.log2(r1_rate_f), axis=1)
mean_unc_r2 = np.nanmean(np.log2(r2_rate_f), axis=1)
ok_unc = np.isfinite(mean_unc_r1) & np.isfinite(mean_unc_r2)
cor_unc = np.corrcoef(mean_unc_r1[ok_unc], mean_unc_r2[ok_unc])[0, 1]
tls_unc, _ = tls(mean_unc_r1[ok_unc], mean_unc_r2[ok_unc])
print(f'Uncorrected correlation: {cor_unc:.4f}')
print(f'Uncorrected TLS 1/slope: {1/tls_unc:.4f}')

X_u = 2 ** mean_unc_r1[ok_unc]
Y_u = 2 ** mean_unc_r2[ok_unc]
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X_u, Y_u, s=8, alpha=0.5)
ax.axline((0, 0), slope=1, linestyle='--', color='gray')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('3-day rate (day$^{-1}$)')
ax.set_ylabel('5-day rate (day$^{-1}$)')
ax.set_title(f'Uncorrected rates\nr = {cor_unc:.3f}')
plt.tight_layout()
plt.show()

---
## 9. Half-life distribution: corrected vs uncorrected

In [ ]:
adj_r1_tune = 2 ** adj_r1_log_tune
adj_r2_tune = 2 ** adj_r2_log_tune

hl_unc   = np.log(2) / np.nanmean(r2_rate_f, axis=1)
hl_corr  = np.log(2) / np.nanmean(adj_r2_tune[keep], axis=1)

ok_hl = np.isfinite(hl_unc) & np.isfinite(hl_corr) & (hl_unc > 0) & (hl_corr > 0)

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=False)
for ax, vals, label in zip(axes, [hl_unc[ok_hl], hl_corr[ok_hl]], ['Uncorrected', 'Corrected']):
    ax.hist(np.log10(vals), bins=30, color='steelblue' if label == 'Uncorrected' else 'black',
            alpha=0.7, edgecolor='white')
    ax.set_xlabel('log10 half-life (days)')
    ax.set_ylabel('# proteins')
    ax.set_title(f'{label} clearance half-life (5-day)')
plt.tight_layout()
plt.show()

---
## 10. Update QQC object and save

In [ ]:
# Combine corrected alpha (Kr) matrices back into miceotope cols order
adj_all = np.full((adj_r1_tune.shape[0], len(miceotope_cols)), np.nan)
for k, col_id in enumerate(days_3_ids):
    adj_all[:, col_idx[col_id]] = adj_r1_tune[:, k]
for k, col_id in enumerate(days_5_ids):
    adj_all[:, col_idx[col_id]] = adj_r2_tune[:, k]

# Replace Alpha_pep with recycling-corrected values
r1_5day_male.miceotopes.Alpha_pep = adj_all

# Collapse to protein level
r1_5day_male = miceotope_protein_collapse(r1_5day_male)

print(f'Alpha_prot shape: {r1_5day_male.miceotopes.Alpha_prot.shape}')

In [ ]:
# Save updated QQC
with open(os.path.join(data_out, 'r1_5day_male.pkl'), 'wb') as f:
    pickle.dump(r1_5day_male, f)
print('Saved updated QQC pickle.')

# Save clearance rate matrices
Alpha_prot = r1_5day_male.miceotopes.Alpha_prot
prot_rows  = r1_5day_male.miceotopes.rows
prot_cols  = r1_5day_male.miceotopes.cols

# Relative (row-normalized)
alpha_norm = normalize_reference_vector(Alpha_prot.copy(), log=True)
pd.DataFrame(alpha_norm, index=prot_rows, columns=prot_cols).to_csv(
    os.path.join(data_out, 'clearance_relative.csv'))

# Absolute
pd.DataFrame(Alpha_prot, index=prot_rows, columns=prot_cols).to_csv(
    os.path.join(data_out, 'clearance_absolute.csv'))

print('Saved clearance_relative.csv and clearance_absolute.csv')